In [1]:
import os
from torchvision import datasets
from PIL import Image
import torchvision.transforms as transforms
import torch
import torchvision.models as models
import numpy as np
import torch.nn as nn

batch_size=20
num_workers=0

data_dir='/floyd/input/kaggle_skin_cancer/'
train_dir=os.path.join(data_dir, 'train/')
valid_dir=os.path.join(data_dir, 'valid/')

train_data_transform=transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_data_transform=transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data=datasets.ImageFolder(train_dir, transform=train_data_transform)
valid_data=datasets.ImageFolder(valid_dir, transform=test_data_transform)

train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         shuffle=True)
valid_loader=torch.utils.data.DataLoader(valid_data,
                                         batch_size=batch_size,
                                         num_workers=num_workers, 
                                         shuffle=True)

loaders_transfer={
    "train": train_loader,
    "valid": valid_loader,
}

In [2]:
model_transfer=models.inception_v3(pretrained=True)
print(model_transfer)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [3]:
print(model_transfer.fc.in_features)

2048


In [4]:
for param in model_transfer.parameters():
    param.requires_grad = True

In [5]:
n_input=model_transfer.fc.in_features
last_layer=nn.Linear(n_input, 2, bias=True)
model_transfer.fc=last_layer
model_transfer.AuxLogits.fc = nn.Linear(768, 2, bias=True)

In [6]:
for param in model_transfer.fc.parameters():
    param.requires_grad = True
for param in model_transfer.AuxLogits.fc.parameters():
    param.requires_grad = True
print(model_transfer)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [7]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()

In [8]:
import torch.optim as optim
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9)

In [9]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def inception_train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            outputs, aux_outputs = model(data)
            loss1 = criterion(outputs, target)
            loss2 = criterion(aux_outputs, target)
            loss = loss1 + 0.4*loss2
            loss.backward()
            optimizer.step()
 
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 100 == 99:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):

            if use_cuda:
                data, target = data.cuda(), target.cuda()
 
            outputs = model(data)
            loss = criterion(outputs, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
            
    return model

In [10]:
model_transfer = inception_train(20, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

Epoch 1, Batch 100 loss: 0.704280
Epoch: 1 	Training Loss: 0.658835 	Validation Loss: 0.355861
Validation loss decreased (inf --> 0.355861).  Saving model ...
Epoch 2, Batch 100 loss: 0.467459
Epoch: 2 	Training Loss: 0.472698 	Validation Loss: 0.332210
Validation loss decreased (0.355861 --> 0.332210).  Saving model ...
Epoch 3, Batch 100 loss: 0.437648
Epoch: 3 	Training Loss: 0.430772 	Validation Loss: 0.300156
Validation loss decreased (0.332210 --> 0.300156).  Saving model ...
Epoch 4, Batch 100 loss: 0.416999
Epoch: 4 	Training Loss: 0.417294 	Validation Loss: 0.287500
Validation loss decreased (0.300156 --> 0.287500).  Saving model ...
Epoch 5, Batch 100 loss: 0.397307
Epoch: 5 	Training Loss: 0.390687 	Validation Loss: 0.260610
Validation loss decreased (0.287500 --> 0.260610).  Saving model ...
Epoch 6, Batch 100 loss: 0.364285
Epoch: 6 	Training Loss: 0.363761 	Validation Loss: 0.246107
Validation loss decreased (0.260610 --> 0.246107).  Saving model ...
Epoch 7, Batch 100 lo

In [11]:
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

<All keys matched successfully>

In [13]:
running_corrects = 0
for batch_idx, (data, target) in enumerate(loaders_transfer['valid']):
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    outputs=model_transfer(data)
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == target.data)
acc = running_corrects.double() / len(loaders_transfer['valid'].dataset)
print(acc)

tensor(0.8970, device='cuda:0', dtype=torch.float64)
